# Constrained optimization

**Table of contents**<a id='toc0_'></a>    
- 1. [In general](#toc1_)    
- 2. [Economic application](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

You will learn to *numerically* solve *constrained* optimization problems (with **scipy.optimize**).

**scipy.optimize:** [overview](https://docs.scipy.org/doc/scipy/reference/optimize.html)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

from types import SimpleNamespace
from copy import deepcopy
import time

from scipy import optimize

## 1. <a id='toc1_'></a>[In general](#toc0_)

Consider the **constrained problem**:

$$
\min_{x_1,x_2,x_3,x_4} x_1x_4(x_1+x_2+x_3) + x_3
$$

subject to

$$
\begin{aligned}
x_1x_2x_3x_4 &\geq 25 \\
x_1^2+x_2^2+x_3^2+x_4^2 &= 40 \\
1 \leq x_1,x_2,x_3,x_4 &\leq 5
\end{aligned}
$$

Define **objective** and **constraints**:

In [2]:
def _objective(x1,x2,x3,x4):
    return x1*x4*(x1+x2+x3)+x3

def objective(x):
    return _objective(x[0],x[1],x[2],x[3])

def ineq_constraint(x):
    return x[0]*x[1]*x[2]*x[3]-25.0 # violated if negative

def eq_constraint(x):
    return 40.0 - np.sum(x**2) # must equal zero

**Chocie of optimizer:** SLSQP

In [3]:
# a. setup
bound = (1.0,5.0)
bounds = (bound, bound, bound, bound)
ineq_con = {'type': 'ineq', 'fun': ineq_constraint} 
eq_con = {'type': 'eq', 'fun': eq_constraint}

# b. call optimizer
x0 = (40**(1/8),40**(1/8),40**(1/8),40**(1/8)) # fit the equality constraint
result = optimize.minimize(objective,x0,
                             method='SLSQP',
                             bounds=bounds,
                             constraints=[ineq_con,eq_con],
                             options={'disp':True})

print('\nx = ',result.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 17.014017289044375
            Iterations: 9
            Function evaluations: 46
            Gradient evaluations: 9

x =  [1.         4.74299968 3.82114992 1.3794083 ]


Manual check of constraints:

In [4]:
print(f'{ineq_constraint(result.x) = }')
assert np.isclose(ineq_constraint(result.x),0.0) or ineq_constraint(result.x) >= 0.0

print(f'{eq_constraint(result.x) = }')
assert np.isclose(eq_constraint(result.x),0.0)

ineq_constraint(result.x) = -1.4382450785888068e-10
eq_constraint(result.x) = -2.0128254618612118e-10


## 2. <a id='toc2_'></a>[Economic application](#toc0_)

Consider the following **consumption-saving problem**:

$$
\begin{aligned}
V(a_0) &= \max_{c_1,c_2,c_3} \frac{c_{1}^{1-\rho}}{1-\rho} + \beta \frac{c_{2}^{1-\rho}}{1-\rho} + \beta^2\frac{c_{3}^{1-\rho}}{1-\rho} + \beta^2\nu\frac{(a_{3}+\kappa)^{1-\rho}}{1-\rho}\\
&\text{s.t.}&\\
m_1 &= (1+r)a_0 + y_1\\
a_1 &= m_1-c_1\\
m_2 &= (1+r)a_1 + y_2\\
a_2 &= m_2-c_2\\
m_3 &= (1+r)a_2 + y_3\\
a_3 &= m_3-c_3\\
c_1,c_2,c_3 &\geq 0\\
a_1,a_2,a_3 &\geq 0\\
\end{aligned}
$$

where 

* $m_t$ is cash-on-hand in period $t\in\{1,2,\dots,T\}$
* $c_t$ is consumption $t$
* $a_t$ is end-of-period assets and income in period $t$
* ${y_t}$ is income in period $t$
* $\beta > 0$ is the discount factor
* $r > -1$ is the interest rate 
* $\rho > 1$ is the CRRA coefficient
* $\nu > 0 $ is the strength of the bequest motive
* $\kappa > 0$ is the degree of luxuriousness in the bequest motive  
* $a_t\geq0$ is a no-borrowing constraint.

**First order conditions:**

* Period 1: If $a_1 > 0$ then $c_1^{-\rho} = \beta(1+r)c_2^{-\rho}$.
* Period 2: If $a_2 > 0$ then $c_2^{-\rho} = \beta(1+r)c_3^{-\rho}$.
* Period 3: If $a_3 > 0$ then $c_3^{-\rho} = \nu(a_3+\kappa)^{-\rho}$.

**Guide to solve such problem:**

1. Setup parameters
2. Formulate objective function
3. Determine how to handle constraints
4. Call optimizer

**Parameters:**

In [5]:
par = SimpleNamespace()
par.a0 = 0.5
par.beta = 0.94
par.r = 0.04
par.rho = 8
par.kappa = 0.5
par.nu = 0.1
par.T = 3
par.y = np.arange(1,par.T+1) # = [1,2,3]

**Objetive function:**

In [6]:
def evaluate(c,par,penalty_factor=10_000):
    """ evaluate model and calculate utility and penalty if constraints are not satisfies """
    
    # a. allocate
    a = np.zeros(par.T) # end-of-period assets
    m = np.zeros(par.T) # cash-on-hand
    cb = np.zeros(par.T) # bounded consumption, defined below
    
    # b. bound consumption and penalty
    penalty = 0.0
    for t in range(par.T): # period-by-period
        
        # i. lagged assets
        a_lag = a[t-1] if t > 0 else par.a0
        
        # ii. cash-on-hand
        m[t] = (1+par.r)*a_lag + par.y[t]
        
        # ii. bound consumption
        if c[t] < 0.0: # too low
            cb[t] = 0.0
            penalty += penalty_factor*np.abs(c[t]-0.0)            
        elif c[t] > m[t]: # too high
            cb[t] = m[t]
            penalty += penalty_factor*np.abs(c[t]-m[t])
        else: # just fine
            cb[t] = c[t]
        
        # d. end-of-period assets 
        a[t] = m[t] - cb[t]
            
    # c. utility
    total_utility = 0.0
    
    # i. consumption
    for t in range(par.T):
        discounting = par.beta**t
        per_period_utility = cb[t]**(1-par.rho)/(1-par.rho)
        total_utility += discounting*per_period_utility
    
    # ii. bequest
    discounting = par.beta**(par.T-1)
    bequest_utility = par.nu*(a[-1]+par.kappa)**(1-par.rho)/(1-par.rho)
    total_utility += discounting*bequest_utility
        
    # d. return
    return total_utility,penalty,m,a
    
def obj(c,par,penalty_factor=10_000):
    """ gateway to evaluate() for optimizer """
    
    utility,penalty,_m,_a = evaluate(c,par,penalty_factor)
    
    return -utility + penalty


**Solve:**

In [7]:
def solve(par,tol=1e-8):
    """ solve model """
    
    # a. initial geuss
    x0 = (par.a0+par.y)/par.T # equal consumption
    
    # b. solve
    t0 = time.time()
    results = optimize.minimize(obj,x0,args=(par,),
                                method='Nelder-Mead',
                                options={'xatol':tol,'fatol':tol,'maxiter':50_000})
    
    if not results.success:
        print(results)
        raise ValueError
    
    print(f'solved model in {time.time()-t0:.3f} secs [nit: {results.nit}, nfev: {results.nfev}]\n')
    
    # show results
    show(par,results)

def show(par,results):
    """ show results """
    
    # final evaluation
    c = results.x
    total_utility,penalty,m,a = evaluate(c,par)
    assert np.isclose(penalty,0.0)

    # print
    print(f't =  0: a = {par.a0:.4f}')
    for t in range(par.T):
        print(f't = {t+1:2d}: y = {par.y[t]:7.4f}, m = {m[t]:7.4f}, c = {c[t]:7.4f}, a = {a[t]:7.4f} ')    
    
    print(f'\ntotal utility = {total_utility:.8f} [penalty = {penalty:.4f}]\n')
    
    # FOC errors
    for t in range(par.T):
        
        if t < par.T-1:
            foc_error = c[t]**(-par.rho) - par.beta*(1+par.r)*c[t+1]**(-par.rho)
        else:
            foc_error = c[t]**(-par.rho) - par.nu*(a[t]+par.kappa)**(-par.rho)
            
        print(f'FOC error in period {t+1:2d}: {foc_error:12.8f}')    

In [8]:
solve(par)

solved model in 0.046 secs [nit: 304, nfev: 558]

t =  0: a = 0.5000
t =  1: y =  1.0000, m =  1.5200, c =  1.5200, a =  0.0000 
t =  2: y =  2.0000, m =  2.0000, c =  2.0000, a =  0.0000 
t =  3: y =  3.0000, m =  3.0000, c =  2.0001, a =  0.9999 

total utility = -0.01039479 [penalty = 0.0000]

FOC error in period  1:   0.03127674
FOC error in period  2:   0.00008937
FOC error in period  3:  -0.00000006


**What happens if the income path is reversed?**

In [9]:
par_alt = deepcopy(par)
par_alt.y = par.y[::-1]
solve(par_alt)

solved model in 0.015 secs [nit: 133, nfev: 249]

t =  0: a = 0.5000
t =  1: y =  3.0000, m =  3.5200, c =  1.9145, a =  1.6055 
t =  2: y =  2.0000, m =  3.6698, c =  1.9090, a =  1.7607 
t =  3: y =  1.0000, m =  2.8312, c =  1.9036, a =  0.9275 

total utility = -0.00540710 [penalty = 0.0000]

FOC error in period  1:  -0.00000000
FOC error in period  2:   0.00000000
FOC error in period  3:  -0.00000000


**Task:** Increase $T$ to 4 where $y_t = 1+t$.

$$
\begin{aligned}
V(a_0) &= \max_{c_1,c_2,\dots c_T} \sum_{t=1}^T \beta^{t-1} \frac{c_{t}^{1-\rho}}{1-\rho} + \beta^{T+1}\nu\frac{(a_{T}+\kappa)^{1-\rho}}{1-\rho}\\
&\text{s.t.}&\\
m_t &= (1+r)a_{t-1} + y_t\\
c_t &\geq 0\\
a_t &\geq 0
\end{aligned}
$$

In [10]:
# write your code here

**Follow-up question:** What is the problem for $T \rightarrow \infty$?

**Note:** We can solve an *intertemporal* problem *as-if* it was a static problem, because there is *no risk*.<br>
For more general problems *with risk*, we need *dynamic optimization*.